In [ ]:
!pip install monai torch

In [19]:
import torch
import numpy as np
import torch.nn as nn
from monai.networks.nets import SwinUNETR
import torch.nn.functional as F
from types import MethodType
import inspect
from typing import Dict, Any, Optional

In [20]:
def _get_device(device: Optional[torch.device] = None) -> torch.device:
    if device is None:
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if isinstance(device, str):
        return torch.device(device)
    return device

def build_swinunetr(params: Dict[str, Any],
                           in_ch: int = 1,
                           out_ch: int = 1,
                           device: Optional[torch.device] = None,
                           wrap_dataparallel: bool = True) -> nn.Module:
    """
    Сборка SwinUNETR из словаря params:
    - фильтрует params по сигнатуре конструктора (чтобы игнорировать лишние ключи);
    - создаёт модель, при наличии >1 GPU оборачивает в DataParallel (если wrap_dataparallel=True);
    - возвращает модель на указанном device.
    """
    device = _get_device(device)
    sig = inspect.signature(SwinUNETR.__init__)
    allowed = {p for p in sig.parameters if p not in ("self", "in_channels", "out_channels")}
    call_kwargs = {k: v for k, v in params.items() if k in allowed}
    model = SwinUNETR(in_ch, out_ch, **call_kwargs)
    if wrap_dataparallel and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    return model.to(device)

def modify_swinunetr_model(model):
    """
    1) заменяет swinViT.patch_embed.proj -> Conv3d(..., kernel=5, stride=2, pad=2, bias=False)
    2) уменьшает число swin стадий на 1 (удаляет layers4)
    3) делает encoder4 новым bottleneck (вместо encoder10) и перестраивает forward, чтобы использовать decoder3..1
    4) добавляет 1x1 адаптеры enc2to3 и enc3to4 при необходимости
    """

    # --------- 1) patch_embed.proj replace ----------
    if not hasattr(model, 'swinViT') or not hasattr(model.swinViT, 'patch_embed'):
        raise AttributeError("model не имеет model.swinViT.patch_embed — проверьте имя атрибута.")
    pe = model.swinViT.patch_embed

    in_chans = getattr(getattr(pe, 'proj', None), 'in_channels', None)
    out_chans = getattr(getattr(pe, 'proj', None), 'out_channels', None)
    embed_dim = getattr(pe, 'embed_dim', None) or out_chans

    new_proj = nn.Conv3d(
        in_channels=int(in_chans),
        out_channels=int(embed_dim),
        kernel_size=5,
        stride=2,
        padding=2,
        bias=False
    )
    pe.proj = new_proj
    print(f"[OK] Заменил patch_embed.proj -> Conv3d({in_chans},{embed_dim},kernel=5,stride=2,pad=2,bias=False)")

    # --------- 2) убрать одну swin-стадию (layers4) ----------
    swin = model.swinViT

    # Найдём layers в порядке
    layers = []
    for name in dir(swin):
        if name.startswith('layers'):
            attr = getattr(swin, name)
            if isinstance(attr, (nn.Module, nn.ModuleList)):
                layers.append((name, attr))

    layers = sorted(layers, key=lambda x: x[0])
    
    
    last_name, last_layer = layers[-1]
    prev_name, prev_layer = layers[-2]
    # удаляем last stage: чтобы forward не падал, оставим placeholder Module 
    # наш forward не будет использовать swinViT.layers4)
    setattr(swin, last_name, nn.Identity())
    # уменьшить num_layers если есть
    if hasattr(swin, 'num_layers'):
        old = int(getattr(swin, 'num_layers'))
        swin.num_layers = max(0, old - 1)
        print(f"[INFO] swinViT.num_layers: {old} -> {swin.num_layers}")
        
    print(f"[OK] Обрезал {last_name} (подставил Identity).")

    # --------- 3) удалить/не использовать encoder10/decoder5 (глубочайший) ----------
    # если они есть — удалим, чтобы избежать путаницы
    removed = []
    for name in ('encoder10', 'decoder5'):
        if hasattr(model, name):
            try:
                delattr(model, name)
            except Exception:
                # если delattr не срабатывает — переопределим как Identity (без изменения forward)
                setattr(model, name, nn.Identity())
            removed.append(name)
    if removed:
        print("[INFO] Удалены/переопределены глубочайшие модули:", ", ".join(removed))

    # --------- 4) добавим 1x1 адаптеры для согласования каналов  ----------
    # нам нужны conv: enc2(48)->enc3(96) и enc3(96)->enc4(192)
    model.enc2to3 = nn.Conv3d(48, 96, kernel_size=1, bias=False)
    nn.init.kaiming_normal_(model.enc2to3.weight, nonlinearity='relu')
    print("[OK] Добавлен enc2to3: Conv3d(48->96, k=1)")

    model.enc3to4 = nn.Conv3d(96, 192, kernel_size=1, bias=False)
    nn.init.kaiming_normal_(model.enc3to4.weight, nonlinearity='relu')
    print("[OK] Добавлен enc3to4: Conv3d(96->192, k=1)")

    # --------- 5) Подменяем forward: использовать encoder1..encoder4 как глубокую часть; encoder4 = bottleneck ----------
    # Новый forward:
    def _new_forward(self, x):
        # x: (B, C=1, D, H, W)
        # encoder1 (full res)
        enc1 = self.encoder1(x)                     # -> 48 ch
        # encoder2 (half)
        e2_in = F.max_pool3d(enc1, kernel_size=2, stride=2)
        enc2 = self.encoder2(e2_in)                 # -> 48 ch
        # encoder3 (quarter) - требуется mapping 48->96
        e3_in = F.max_pool3d(enc2, kernel_size=2, stride=2)
        e3_in_mapped = self.enc2to3(e3_in)         # -> 96 ch
        enc3 = self.encoder3(e3_in_mapped)         # -> 96 ch
        # encoder4 (eighth) - mapping 96->192
        e4_in = F.max_pool3d(enc3, kernel_size=2, stride=2)
        e4_in_mapped = self.enc3to4(e4_in)         # -> 192 ch
        enc4 = self.encoder4(e4_in_mapped)         # -> 192 ch (новый bottleneck)

        d3 = self.decoder3(enc4, enc3)             # in=192 -> out=96, skip enc3 (96)
        d2 = self.decoder2(d3, enc2)               # in=96  -> out=48, skip enc2 (48)
        d1 = self.decoder1(d2, enc1)               # in=48  -> out=48, skip enc1 (48)

        out = self.out(d1)
        return out

    # Bind new forward to model instance
    model.forward = MethodType(_new_forward, model)
    print("[OK] Подменён model.forward: использую encoder1..encoder4 в качестве 3 уровней + bottleneck=encoder4; "
          "декодеры используются decoder3->decoder2->decoder1.")

    return model

In [21]:
def load_checkpoint(model, checkpoint_path, device):
  checkpoint = torch.load(checkpoint_path, map_location=device)
  checkpoint_dict = {}
  for key, value in checkpoint["model_state_dict"].items():
      if key.startswith('module.'):
          key = key.replace('module.', '')
      checkpoint_dict[key] = value
  
  model.load_state_dict(checkpoint_dict)
  return model

In [22]:
model_path = "/home/burlaka-na/Downloads/swinunetr_tiny_after_faultseg3d.pth"

MODEL_PARAMS = dict(
    patch_size=(2, 2, 2),
    depths=(2, 2, 2, 1),
    num_heads=(3, 6, 12, 24),
    window_size=(7, 7, 7),
    qkv_bias=True,
    mlp_ratio=4.0,
    feature_size=48,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    dropout_path_rate=0.1,
    patch_norm=True,
    spatial_dims=3,
)
MAIN_DEVICE = "cpu"

model = build_swinunetr(MODEL_PARAMS, in_ch=1, out_ch=1)

swin_tiny_model = modify_swinunetr_model(model)

swin_tiny_model = swin_tiny_model.to(MAIN_DEVICE)

swin_tiny_model = load_checkpoint(swin_tiny_model, model_path, MAIN_DEVICE)

device = next(model.parameters()).device

x = torch.randn(1, 1, 128, 128, 128, device=device)

out = swin_tiny_model(x)

print(out.shape)

[OK] Заменил patch_embed.proj -> Conv3d(1,48,kernel=5,stride=2,pad=2,bias=False)
[INFO] swinViT.num_layers: 4 -> 3
[OK] Обрезал layers4 (подставил Identity).
[INFO] Удалены/переопределены глубочайшие модули: encoder10, decoder5
[OK] Добавлен enc2to3: Conv3d(48->96, k=1)
[OK] Добавлен enc3to4: Conv3d(96->192, k=1)
[OK] Подменён model.forward: использую encoder1..encoder4 в качестве 3 уровней + bottleneck=encoder4; декодеры используются decoder3->decoder2->decoder1.
torch.Size([1, 1, 128, 128, 128])
